In [ ]:
import pandas as pd
import numpy as np
import cupy
import cudf
import matplotlib.pyplot as plt, gc, os

targets_dir = '../input/amex-default-prediction/train_labels.csv'
train_dir   = '../input/amex-data-integer-dtypes-parquet-format/train.parquet'

I was investigating the implications of N/A values in the data. I found some interesting dependencies. However, I am not really sure how to implement the knowlegde into my model.

Maybe this is helpful to someone. And maybe someone will be can be helpful to me:)

In [ ]:
# Readin funtion
def read_file(path, usecols = None, na = True):
    
    print('Reading data...')
    
    # Distinguish for special columns if needed
    if usecols is not None: df = cudf.read_parquet(path, columns=usecols)
    else:                   df = cudf.read_parquet(path)
    
    # Reduce cus_id
    unique_cus_ids = df.customer_ID.unique().to_pandas().values
    assignment     = dict(zip(unique_cus_ids, list(range(len(unique_cus_ids)))))
    df.customer_ID = df.customer_ID.to_pandas().apply(lambda x: assignment[x]).astype('int32')
    
    del unique_cus_ids, assignment
    gc.collect()

    # Reduce dtype for 'S_2'
    df.S_2 = cudf.to_datetime( df.S_2 )
    
    # fill missing data
    if na: df = df.fillna(NAN_VALUE)
    
    print('shape of data:', df.shape)
    
    return df

In [ ]:
#Load data for the N/A value analysis
test    = read_file(train_dir, na = False)
targets = cudf.read_csv(targets_dir)

targets['target'] = targets.target.astype('int8')
targets.drop('customer_ID', axis=1, inplace = True)

In [ ]:
# Build a summary table for the N/A values
summary = pd.DataFrame()

# Which feature has how many N/A values?
summary['Feature']        = test.columns.values
summary['number_of_N/As'] = test.isna().sum().to_pandas().values

# Calculate the default rate among customers with N/A values
# 'avgs' are the default rates for by features
# 'na_cus' is the number of customers displaying N/A values
def_rates_no_na_cus, def_rates_na_cus, no_na_cus, na_cus = [], [], [], []

for feature in test.columns.values:
    
    
    # Get customers ids WITHOUT N/A values
    cus  = test.loc[test[f'{feature}'].notna()].customer_ID.unique().values
    
    # If there are no such customers than there is no default rate among them    
    if len(cus) == 0:
        def_rates_no_na_cus.append(0)
        no_na_cus.append(0)
    else: 
        avg = targets.iloc[cus].mean().values[0]
        def_rates_no_na_cus.append(float(avg))
        no_na_cus.append(len(cus))
        
    
    # Get customers ids WITH N/A values
    cus1 = test.loc[test[f'{feature}'].isna()].customer_ID.unique().values
    
    # If there are no such customers than there is no default rate among them
    if len(cus1) == 0:
        def_rates_na_cus.append(0)
        na_cus.append(0)  
    else:
        avg = targets.iloc[cus1].mean().values[0]
        def_rates_na_cus.append(float(avg))
        na_cus.append(len(cus1))
        

summary['Cus_with_N/As']        = na_cus
summary['Def_rate_na_cus']      = def_rates_na_cus
summary['Cus_without_N/As']     = no_na_cus
summary['Def_rates_no_na_cus']  = def_rates_no_na_cus

Here you go! This is the table!

In [ ]:
summary

In [ ]:
# Avarage default rate
avg_def_rate = targets.target.mean()

Note that there is a number of feature where customers with N/A values display a significantly high default rates.



In [ ]:
cols = ['Feature', 'number_of_N/As', 'Cus_with_N/As', 'Def_rate_na_cus']
signi_def_rate = avg_def_rate + 0.03
summary.sort_values('Def_rate_na_cus' ,ascending = False).loc[summary.Def_rate_na_cus > signi_def_rate, cols]

In [ ]:
cols = ['Feature', 'number_of_N/As', 'Cus_with_N/As', 'Def_rate_na_cus']
signi_def_rate = avg_def_rate - 0.03
summary.sort_values('Def_rate_na_cus' ,ascending = True).loc[summary.Def_rate_na_cus.between(0.01, signi_def_rate) , cols]

This effect is even stronger when considering groups of customers with have no N/A values for some features


In [ ]:
cols = ['Feature', 'number_of_N/As', 'Cus_without_N/As', 'Def_rates_no_na_cus']
signi_def_rate = avg_def_rate + 0.03
summary.sort_values('Def_rates_no_na_cus' ,ascending = False).loc[summary.Def_rates_no_na_cus > signi_def_rate, cols]

In [ ]:
cols = ['Feature', 'number_of_N/As', 'Cus_without_N/As', 'Def_rates_no_na_cus']
signi_def_rate = avg_def_rate - 0.03
summary.sort_values('Def_rates_no_na_cus' ,ascending = True).loc[summary.Def_rates_no_na_cus.between(0.01, signi_def_rate) > signi_def_rate, cols]

1) Is there a way to make sense of of this?

2) Currently, I am using gxboost. Using dummies to such customers is not really helpful as those groups tend to be rather small and in the algorithm the given feature will likely not be used to split the tree. Is there some way I still can implement this knowledge into my model?
